In [5]:
import pandas as pd
true_df=pd.read_csv('/content/True.csv')
fake_df=pd.read_csv('/content/Fake.csv')
true_df['label']=1
fake_df['label']=0
df=pd.concat([true_df,fake_df],ignore_index=True)
df=df[['title','text','label']]
df.dropna(inplace=True)
df['content']=df['title']+""+df['text']
df=df[['content','label']]
df.head()


,content,label
0,"As U.S. budget fight looms, Republicans flip t...",1
1,U.S. military to accept transgender recruits o...,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,1
3,FBI Russia probe helped by Australian diplomat...,1
4,Trump wants Postal Service to charge 'much mor...,1


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix
import re

def clean_text(text):
  text=text.lower()
  text=re.sub(r'/w','',text)
  text=re.sub(r'/s+[a-zA-Z]\s+','',text)
  text=re.sub(r'\s+','',text)
  return text

df['content']=df['content'].apply(clean_text)
df.head()

,content,label
0,"asu.s.budgetfightlooms,republicansfliptheirfis...",1
1,u.s.militarytoaccepttransgenderrecruitsonmonda...,1
2,senioru.s.republicansenator:'letmr.muellerdohi...,1
3,fbirussiaprobehelpedbyaustraliandiplomattip-of...,1
4,trumpwantspostalservicetocharge'muchmore'foram...,1


In [11]:
x=df['content']
y=df['label']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
tfidf=TfidfVectorizer(max_features=5000)
x_train_tfidf=tfidf.fit_transform(x_train)
x_test_tfidf = tfidf.transform(x_test)

model=LogisticRegression()
model.fit(x_train_tfidf,y_train)
y_pred=model.predict(x_test_tfidf)
print("confusion matrix:\n",confusion_matrix(y_test,y_pred))
print("\nclassification report:\n",classification_report(y_test,y_pred))

def check_news(news):
  news=clean_text(news)
  vector = tfidf.transform([news])
  result=model.predict(vector)
  return "Real" if result[0]==1 else "Fake"
print(check_news("Government confirms new economic reforms"))


confusion matrix:
 [[4641    9]
 [  13 4317]]

classification report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      4650
           1       1.00      1.00      1.00      4330

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980

Fake
